In [13]:
import pandas as pd
import numpy as np

In [14]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [15]:
dep_rates = pd.read_csv('C:\\my\\prog\\pet banks\\raw_data\\no regions\\deposit_rates.csv', sep=',', encoding='utf-8')
monetary_rew = pd.read_csv('C:\\my\\prog\\pet banks\\raw_data\\no regions\\monetary_review.csv', sep=',', encoding='utf-8')
money_mass = pd.read_csv('C:\\my\\prog\\pet banks\\raw_data\\no regions\\money_mass.csv', sep=',', encoding='utf-8')

In [16]:
monetary_rew = monetary_rew[monetary_rew["Монетарный обзор по банкам"] != "Кредиты"]

In [17]:
def common_clean_data(df, delete_second_rows):
    df = df.copy()
    df.replace('-', np.nan, inplace=True)
    if delete_second_rows:
         df = pd.concat([df.iloc[:, 0], df.iloc[:, 1::2]], axis=1)
    df.columns = df.iloc[1]
    df.columns.values[0] = 'value'
    df.drop(index=1, inplace=True)
    df.dropna(how='any', inplace=True)
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda row: pd.to_numeric(row, errors='raise'))
    df.reset_index(drop=True, inplace=True)
    return df

In [18]:
dep_rates_cleaned = common_clean_data(dep_rates, True)
monetary_rew_cleaned = common_clean_data(monetary_rew, False)
money_mass_cleaned = common_clean_data(money_mass, False)

In [19]:
def to_long(df):
    return df.melt(id_vars="value",var_name="month",value_name="rate")

dep_rates_long = to_long(dep_rates_cleaned)
monetary_rew_long = to_long(monetary_rew_cleaned)  
money_mass_long = to_long(money_mass_cleaned)

In [20]:
context_df = pd.concat([dep_rates_long, monetary_rew_long, money_mass_long], ignore_index=True)

In [21]:
context_df['month'] = pd.to_datetime(
    context_df['month'].str.strip(),
    format='%m.%y'
)
context_df['rate'] = pd.to_numeric(context_df['rate'], errors='raise')


In [22]:
context_df = context_df.pivot_table(index='month', columns='value', values='rate').reset_index()

In [26]:
context_df = context_df.dropna()

In [27]:
context_df.to_csv('C:\\my\\prog\\pet banks\\clean_data\\contextKZ.csv', index=False,)